In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib 
import math
from ipywidgets import interact, IntSlider, Dropdown, VBox, Output
from matplotlib.animation import FuncAnimation, HTMLWriter, FFMpegWriter
import itertools
import datetime
import itertools
import numpy as np
import plotly.graph_objects as go
from IPython.display import HTML

In [ ]:
# Colors

color_BF = 'purple'
color_extremal = 'orange'
color_neuron = 'blue'
color_neuron_pos = 'green'
color_neuron_neg = 'red'
color_boundaries = 'black'

In [ ]:
def relu(x):
    return np.maximum(0, x)

def indicator(condition):
    return condition.astype(int)

In [ ]:
# 2-Lyaer ReLu Neural Network
def twoLayerReluNet(a, w, x):
    return np.sum(a[:, np.newaxis] * relu(np.inner(w,x)), axis=0)

def loss(a, w, x, y):
    return (1/(2*n)) * np.sum((y - twoLayerReluNet(a, w, x))**2)

In [ ]:
## Typical parameter gradients
def gradient_flow(a,w):
    
    relu_mask = (np.inner(w,x) > 0).astype(float)
    Dt = - 1/n * np.sum(((twoLayerReluNet(a, w, x) - y) * relu_mask)[:, :, np.newaxis] * x, axis=1) # Not sure about this one here

    grad_a = np.einsum('ij,ij->i', Dt, w) 
    grad_w = Dt * a[:, np.newaxis]
 
    return grad_a, grad_w

def generate_initial_data(lambda_param,m):
    mean = np.zeros(d)  # Mean vector with all zeros
    cov = np.eye(d)     # Covariance matrix as the identity matrix
    g = np.random.multivariate_normal(mean, cov, size=m)  # Generate n samples from the normal distribution
    w = lambda_param * g

    w_norms = np.linalg.norm(w, axis=1)
    s = np.random.choice([-1, 1], size=m)    
    a = w_norms * s

    return a, w, s

def generate_set_Splus_Sminus(s,w,m):
    # Compute dot products ⟨w_j^0, x_k⟩ for all j and k
    dot_products = np.inner(w,x)  # Shape (m, n)
    
    # Conditions
    pos_k_mask = y > 0  # Indices where y_k > 0
    neg_k_mask = y < 0  # Indices where y_k < 0

    Splus = np.array([i for i in range(m) if s[i] == 1 ])
    Sminus = np.array([i for i in range(m) if s[i] == -1] )

    return Splus, Sminus

In [ ]:
def gradient_descent(learning_rate, steps, lambda_param,m):

    a, w, s = generate_initial_data(lambda_param,m)
    sPlus, sMinus = generate_set_Splus_Sminus(s,w,m)

    np.save('weights\\s_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, steps), s)
    np.save('weights\\sPlus_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, steps), sPlus)
    np.save('weights\\sMinus_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, steps), sMinus)

    a_values = []
    w_values = []

    #for n in range(math.ceil(time/step_size)):
    for n in range(steps): 
        grad_a, grad_w = gradient_flow(a,w)
        a = a + learning_rate * grad_a
        w = w + learning_rate * grad_w

        # Save the current values of a, w, and b
        a_values.append(a.copy())
        w_values.append(w.copy())

    return a_values, w_values

In [ ]:
def mathcalD(w, theta_a, theta_w):
    relu_mask = (np.inner(w,x) > 0).astype(float)
    Dt = - 1/n * np.sum(((twoLayerReluNet(theta_a, theta_w, x) - y) * relu_mask)[:, :, np.newaxis] * x, axis=1) 
    return Dt

def neuron_activation(w):
    return np.sign(np.inner(w,x))
    
def mathcalD_u(u):
    relu_mask = (u > 0).astype(float)
    Dt = 1/n * np.sum((y * relu_mask)[:,:, np.newaxis] * x, axis=1) 
    return Dt

def mathcalD_u_non_zero(u, theta_a, theta_w):
    relu_mask = (u > 0).astype(float)
    Dt = - 1/n * np.sum(((twoLayerReluNet(theta_a, theta_w, x) - y) * relu_mask)[:, :, np.newaxis] * x, axis=1) 
    return Dt

def funcG(w):
    zero_w = np.zeros([1,d])
    zero_a = np.zeros([1])
    
    return np.inner(w,mathcalD(w, zero_a, zero_w))

def funcG_non_zero(w, theta_a, theta_w):
    return np.inner(w,mathcalD(w, theta_a, theta_w))



def is_extremal_vector(u, coord):
    x_coord, y_coord = coord
    return all(u[i] * (x_coord * x[i, 0] + y_coord * x[i, 1]) >= 0 for i in range(len(x)))

def is_extremal_vector_non_zero(u, coord):
    return 0 

In [ ]:
def mathcalD_u_non_zero(u, theta_a, theta_w):
    relu_mask = (u > 0).astype(float)
    Dt_unnormalized = np.array([np.sum(((twoLayerReluNet(theta_a, theta_w, x) - y) * relu_mask)[:,np.newaxis] * x, axis=0)])

    if np.any(u == 0) and not np.any(np.all(Dt_unnormalized == 0, axis=1)):
        Dt_unnormalized = gen_set_Dt(Dt_unnormalized, u, 0, theta_a, theta_w)
    Dt = - 1/n * Dt_unnormalized
    return Dt

def gen_set_Dt(Dt_unnormalized, u, k, theta_a, theta_w):
    for i in range(n):
        if(u[i] == 0):
            for vec in Dt_unnormalized:
                new_vec = vec + (twoLayerReluNet(theta_a, theta_w, x) - y)[i] * x[i]
                Dt_unnormalized = np.append(Dt_unnormalized, [new_vec], axis=0)
    return Dt_unnormalized

def perp_vector(v):
    """
    Returns the normalized perpendicular vector(s) for input vector(s) v.
    v can be a 1D array of shape (2,) or a 2D array of shape (n, 2).
    """
    v = np.asarray(v)
    if v.ndim == 1:
        if np.all(v == 0):
            return np.array([0, 0])
        perp = np.array([-v[1], v[0]])
        return perp / np.linalg.norm(perp)
    elif v.ndim == 2:
        perp = np.empty_like(v)
        norms = np.linalg.norm(np.stack([-v[:,1], v[:,0]], axis=1), axis=1)
        # Avoid division by zero
        zero_mask = norms == 0
        perp[~zero_mask] = np.stack([-v[~zero_mask,1], v[~zero_mask,0]], axis=1) / norms[~zero_mask, np.newaxis]
        perp[zero_mask] = 0
        return perp
    else:
        raise ValueError("Input must be a 1D or 2D numpy array of shape (2,) or (n,2)")

# def extremal_vectors_time(theta_a, theta_w, index):
#     permutations = list(itertools.product([1, 0, -1], repeat=n))
#     vectors = np.empty((0, d))
#     for perm in permutations:
#         u = np.array(perm)
#         Dt_vectors = mathcalD_u_non_zero(u, theta_a, theta_w)

#         zeros_indexes = np.where(u == 0)[0]
#         one_indexes = np.where(u == 1)[0]
#         minus_one_indexes = np.where(u == -1)[0]

#         # Since we are in 2D it is only possible for an extremal vectors to have at most one zero for its activation
#         # We want at least one 1 value activation to avoid trivial cases
#         if len(zeros_indexes) == 1 and len(one_indexes) != 0: 
#             zero_index = zeros_indexes[0]

#             innerproductVectors = np.inner(x[zero_index], Dt_vectors)

#             if len(one_indexes) == 0:
#                 minus_one_inner_product = np.dot(x[minus_one_indexes],x[zero_index])
#                 activation_cone_exists = np.all(minus_one_inner_product <= 0) or np.all(minus_one_inner_product >= 0)
#             elif len(minus_one_indexes) == 0:
#                 plus_one_inner_product = np.dot(x[one_indexes],x[zero_index] )
#                 activation_cone_exists = np.all(plus_one_inner_product <= 0) or np.all(plus_one_inner_product >= 0)
#             else:
#                 minus_one_inner_product = np.dot(x[minus_one_indexes],x[zero_index])
#                 plus_one_inner_product = np.dot(x[one_indexes],x[zero_index] )
#                 activation_cone_exists =  (np.all(minus_one_inner_product <= 0) and np.all(plus_one_inner_product >= 0)) or (np.all(minus_one_inner_product >= 0) and np.all(plus_one_inner_product <= 0))
                
#             some_extremal_vector_exists = (not np.all(innerproductVectors > 0) and not np.all(innerproductVectors < 0)) or (np.any(innerproductVectors == 0))

#             if some_extremal_vector_exists and activation_cone_exists:
#                 perp_vec = np.array([1, -x[zero_index][0]/x[zero_index][1]])
#                 # We want to add the vector perpendicular to the one with activation 0
#                 # As well as the same vector with opposite sign
#                 vectors = np.append(vectors, perp_vec[np.newaxis, :], axis=0)
#                 # vectors = np.append(vectors, -perp_vec[np.newaxis, :], axis=0)
#                 print(f"Adding extremal vector for u: {u}")

#         elif len(zeros_indexes) == 0: # Then Dt is a single vector
#             x_coord, y_coord = Dt_vectors[0]
#             if all(u[i] * (x_coord * x[i, 0] + y_coord * x[i, 1]) > 0 for i in range(len(x))) or all(u[i] * (x_coord * x[i, 0] + y_coord * x[i, 1]) < 0 for i in range(len(x))):
#                 if x_coord != 0 or y_coord != 0:
#                     # We want to add the vector perpendicular to the one with activation 0
#                     # As well as the same vector with opposite sign
#                     vectors = np.append(vectors, Dt_vectors[0][np.newaxis, :], axis=0)
#                     # vectors = np.append(vectors, -Dt_vectors[0][np.newaxis, :], axis=0)
#                 print(f"Adding extremal vector for u: {u} with coordinates {x_coord}, {y_coord} at step {index}")

#     return vectors

def extremal_vectors_time(theta_a, theta_w):
    permutations = list(itertools.product([1, 0, -1], repeat=n))
    vectors = np.empty((0, d))
    eps = 1e-10  # Numerical tolerance

    for perm in permutations:
        u = np.array(perm)
        Dt_vectors = mathcalD_u_non_zero(u, theta_a, theta_w)

        zeros_indexes = np.where(u == 0)[0]
        one_indexes = np.where(u == 1)[0]
        minus_one_indexes = np.where(u == -1)[0]

        # Case 1: Exactly one zero activation, and at least one +1 activation
        if len(zeros_indexes) == 1 and len(one_indexes) > 0:
            zero_index = zeros_indexes[0]
            x0 = x[zero_index]

            innerproductVectors = np.inner(x[zero_index], Dt_vectors)

            # Perpendicular vector to x0 (i.e., lies in the hyperplane)
            v_perp = np.array([-x0[1], x0[0]])  # 90° rotation

            # Dot product of all other normals with v_perp
            plus_dot = x[one_indexes] @ v_perp
            minus_dot = x[minus_one_indexes] @ v_perp

            # Check that all plus ones lie on one side, and all minus ones on the other
            activation_cone_exists = (
                np.all(plus_dot > 0) and np.all(minus_dot < 0)
            ) or (
                np.all(plus_dot < 0) and np.all(minus_dot > 0)
            )

            # # Check if Dt_vectors cross the hyperplane (not all on one side)
            # Dt_projection = np.inner(Dt_vectors, v)
            # some_extremal_vector_exists = (
            #     np.any(Dt_projection > eps) and np.any(Dt_projection < -eps)
            # )

            some_extremal_vector_exists = (not np.all(innerproductVectors > 0) and not np.all(innerproductVectors < 0)) or (np.any(innerproductVectors == 0))

            if activation_cone_exists and some_extremal_vector_exists:
                # Construct a vector perpendicular to v (i.e. defining the hyperplane)
                # Normal vector to the hyperplane
                k = x[zero_index]

                # Project Dt_vectors onto the hyperplane: remove the component along n
                n_norm_sq = np.dot(k, k)
                Dt_proj = Dt_vectors - (Dt_vectors @ k[:, None]) * k[None, :] / n_norm_sq

                # Average the directions to get a stable representative vector
                mean_vec_in_plane = np.mean(Dt_proj, axis=0)

                # Normalize direction to avoid scaling issues
                if np.linalg.norm(mean_vec_in_plane) > eps:
                    perp_vec = mean_vec_in_plane / np.linalg.norm(mean_vec_in_plane)
                    vectors = np.append(vectors, perp_vec[np.newaxis, :], axis=0)

                # We want to add the vector perpendicular to the one with activation 0
                # As well as the same vector with opposite sign
                vectors = np.append(vectors, perp_vec[np.newaxis, :], axis=0)

        # Case 2: No zeros — Dt is a single vector, so just check alignment
        elif len(zeros_indexes) == 0 and Dt_vectors.shape[0] == 1:
            v = Dt_vectors[0]
            signs = [u[i] * np.dot(v, x[i]) for i in range(len(x))]
            
            if all(s > eps for s in signs) or all(s < -eps for s in signs):
                if not np.allclose(v, 0):
                    v_normalized = v / np.linalg.norm(v)
                    vectors = np.append(vectors, v_normalized[np.newaxis, :], axis=0)

    return vectors


# Training

In [ ]:
def train(n_steps, learning_rate, lambda_param,m):
    a_values, w_values = gradient_descent(learning_rate, n_steps, lambda_param,m)

    # Convert lists to arrays for easier saving and plotting
    a_values = np.array(a_values)
    w_values = np.array(w_values)

    # Save the arrays to disk
    np.save('weights\\a_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps), a_values)
    np.save('weights\\w_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps), w_values)

    print(f'Completed training for steps: {n_steps}, learning rate: {learning_rate}, m value: {m}')

d = 2

x = np.array([[1,1],[3,1]])
y = np.array([-1,2])


n = len(y)

m = 60
lambda_param = 1e-7
learning_rate = 1e-3
n_steps = 30500

train(n_steps, learning_rate, lambda_param,m)

# Critical Points and Extremal Vectors

In [ ]:
import plotly.graph_objects as go

current_a_values = np.load('weights\\a_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))
current_w_values = np.load('weights\\w_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))

# Generate points on a 2D circle
theta = np.linspace(0, 2 * np.pi, 400)
x_plot = np.cos(theta)
y_plot = np.sin(theta)

line_styles = ['dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
max_radius = 3
min_radius = -0.7

fig = go.Figure()

# Apply the function to each point
z = np.zeros(len(x_plot))
for i in range(len(x_plot)):
    z[i] = funcG(np.array([[x_plot[i], y_plot[i]]]))

def make_frame(index, pad_to=None):
    a = current_a_values[index]
    w = current_w_values[index]

    # Apply the function to each point
    z = np.zeros(len(x_plot))
    for i in range(len(x_plot)):
        z[i] = funcG_non_zero(np.array([[x_plot[i], y_plot[i]]]), a, w)

    traces = []

    r_circle = [0.005] * 361  # cannot use exactly 0 since plotly makes this a point
    theta_circle = np.linspace(0, 360, 361)

    traces.append(go.Scatterpolar(
        r=r_circle,
        theta=theta_circle,
        mode='lines',
        line=dict(color='black', width=3, dash='dot'),
        name='Radius=0',
        showlegend=True,
        opacity= 1.0  # Lower opacity for the circle
    ))

    # BF Function
    traces.append(go.Scatterpolar(
        r = z,
        theta = theta * 180 / np.pi,  # Convert radians to degrees for polar plot
        mode = 'lines',
        name = 'BF Function',
        line=dict(color=color_BF, width=2),
        showlegend=True
    ))

    # Boundary Vectors
    l = 1
    for idx, vec in enumerate(x):
        style = line_styles[idx % len(line_styles)]

        

        x_line_orth,y_line_orth = vec[0], vec[1]

        # Generate x1 
        x1 = 1
        # Calculate corresponding x2 values
        x2 = - x_line_orth  / y_line_orth
        
        # Compute angle theta in degrees
        theta_line = np.degrees(np.arctan2(x2, x1))

        # Define radii for the line (e.g., from 0 to max radius in your plot)
        r_line = np.linspace(min_radius, max_radius, 100)

        # Line in original direction
        theta_line_array = np.full_like(r_line, fill_value=theta_line)

        # Line in opposite direction (theta + 180 degrees)
        theta_opposite_array = np.full_like(r_line, fill_value=(theta_line + 180) % 360)

        # Add line traces
        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_line_array,
            mode='lines',
            line=dict(color=color_boundaries, width=0.5, dash=style),
            name=f'$w^T x_{l} = 0$'
        ))
        l+=1

        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_opposite_array,
            mode='lines',
            line=dict(color=color_boundaries, width=0.5, dash=style),
            showlegend=False
        ))

    # Extremal vectors
    legend = True # Only add extremal vectors once
    extremal_vectors = extremal_vectors_time(a, w)
    for vec in extremal_vectors:
        x1,x2 = vec[0], vec[1]
        
        # Compute angle theta in degrees
        theta_line = np.degrees(np.arctan2(x2, x1))

        # Define radii for the line (e.g., from 0 to max radius in your plot)
        r_line = np.linspace(min_radius, max_radius, 100)

        # Line in original direction
        theta_line_array = np.full_like(r_line, fill_value=theta_line)

        # Line in opposite direction (theta + 180 degrees)
        theta_opposite_array = np.full_like(r_line, fill_value=(theta_line + 180) % 360)

        # Add line traces
        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_line_array,
            mode='lines',
            line=dict(color=color_extremal, width=2.5),
            name=f'Extremal vector',
            showlegend=legend
        ))
        legend = False

    # Padding to consistent trace count
    if pad_to:
        while len(traces) < pad_to:
            traces.append(go.Scatterpolar(r=[], theta=[], mode='lines', showlegend=False))

    
    return traces

max_trace_count = 80  # Set a maximum trace count for padding
# Create frames
frame_indices = np.linspace(0, len(current_a_values)-1, 500, dtype=int)  # 50 frames for speed
frames = [
    go.Frame(
        data=make_frame(idx, pad_to=max_trace_count),
        name=str(idx)
    )
    for idx in frame_indices
]

# Initial data
initial_data = make_frame(frame_indices[0], pad_to=max_trace_count)

# After creating initial_data and frames

fig.update_layout(
    title=dict(
            text='Crtical point and extremal vectors',
            x=0.5,           # Center the title
            xanchor='center' # Anchor at the center
        ),
    showlegend = True,
    polar=dict(
        radialaxis=dict(
            range=[min(z)*1.1, max(z)*1.1],  # Change this to your desired max radius
            showticklabels = False, 
            visible = True,
            gridcolor = 'gray',
            showgrid = True,
        ),
        angularaxis = dict(
            showticklabels = False,
            gridcolor = 'gray',
            showgrid = False
        )
    )
)

# Create figure
fig = go.Figure(
    data=initial_data,
    layout=go.Layout(
        # title=dict(
        #     text='Crtical point and extremal vectors',
        #     x=0.5,           # Center the title
        #     xanchor='center' # Anchor at the center
        # ),
        showlegend=True,
        polar=dict(
            radialaxis=dict(
                range=[min_radius, max_radius],  # Change this best suited radius
                showticklabels=False,
                visible=True,
                gridcolor='gray',
                showgrid=True,
            ),
            angularaxis=dict(
                showticklabels=False,
                gridcolor='gray',
                showgrid=False
            )
        ),
        margin=dict(l=20, r=20, t=60, b=20),
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]},
                {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }],
        sliders=[{
            "steps": [
                {"args": [[str(idx)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                 "label": str(idx),
                 "method": "animate"}
                for idx in frame_indices
            ],
            "transition": {"duration": 0},
            "x": 0.1,
            "len": 0.9,
            "currentvalue": {"visible": True},  # Hide the current value number
            "tickwidth": 0,                      # Hide the ticks
            "tickcolor": "rgba(0,0,0,0)",        # Make ticks transparent (optional)
        }]
    ),
    frames=frames
)


fig.write_html("plotly_figures/Crtical_points_Extremal_vectors_animation.html", include_plotlyjs="cdn",include_mathjax="cdn", auto_play=False)
HTML(fig.to_html(include_plotlyjs="cdn",include_mathjax="cdn"))

# Full Model

In [ ]:
import plotly.graph_objects as go
import numpy as np

current_a_values = np.load('weights\\a_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))
current_w_values = np.load('weights\\w_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))

sPlus = np.load('weights\\sPlus_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))
sMinus = np.load('weights\\sMinus_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))

# --- Prepare data and frame indices ---
x1_range = np.linspace(-5, 5, 200)
x_input = np.stack([x1_range, np.ones_like(x1_range)], axis=1)
theta = np.linspace(0, 2 * np.pi, 400)
x_plot = np.cos(theta)
y_plot = np.sin(theta)
line_styles = ['dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
max_radius = 3
min_radius = -0.7

frame_indices = np.linspace(0, len(current_a_values)-1, 500, dtype=int)

def rescale_magnitudes(w, r_min=0.2, r_max=0.8):
    """
    Rescale the norms of w to be between r_min and r_max for polar plotting.
    """
    w_norms = np.linalg.norm(w, axis=1)
    
    return w_norms * 2.25

def make_polar_traces(a, w):
    # Polar plot traces
    z = np.zeros(len(x_plot))
    for i in range(len(x_plot)):
        z[i] = funcG_non_zero(np.array([[x_plot[i], y_plot[i]]]), a, w)

    traces = []

    r_circle = [0.005] * 361
    theta_circle = np.linspace(0, 360, 361)
    traces.append(go.Scatterpolar(
        r=r_circle,
        theta=theta_circle,
        mode='lines',
        line=dict(color='black', width=3, dash='dot'),
        name='Radius=0',
        showlegend=True,
        opacity=1.0,
        subplot='polar'
    ))

    traces.append(go.Scatterpolar(
        r=z,
        theta=theta * 180 / np.pi,
        mode='lines',
        name='BF Function',
        line=dict(color=color_BF, width=2),
        subplot='polar'
    ))

    l = 1
    for idx, vec in enumerate(x):
        style = line_styles[idx % len(line_styles)]
        x_line_orth, y_line_orth = vec[0], vec[1]
        x1 = 1
        x2 = - x_line_orth / y_line_orth
        theta_line = np.degrees(np.arctan2(x2, x1))
        r_line = np.linspace(min_radius, max_radius, 100)
        theta_line_array = np.full_like(r_line, fill_value=theta_line)
        theta_opposite_array = np.full_like(r_line, fill_value=(theta_line + 180) % 360)
        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_line_array,
            mode='lines',
            line=dict(color=color_boundaries, width=0.8, dash='dash'),
            subplot='polar',
            showlegend=False
        ))
        l += 1
        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_opposite_array,
            mode='lines',
            line=dict(color=color_boundaries, width=0.8, dash='dash'),
            showlegend=False,
            subplot='polar',
        ))

    legend = True
    extremal_vectors = extremal_vectors_time(a, w)
    for vec in extremal_vectors:
        x1, x2 = vec[0], vec[1]
        theta_line = np.degrees(np.arctan2(x2, x1))
        r_line = np.linspace(min_radius, max_radius, 100)
        theta_line_array = np.full_like(r_line, fill_value=theta_line)
        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_line_array,
            mode='lines',
            line=dict(color=color_extremal, width=2.5),
            name=f'Extremal vector',
            showlegend=legend,
            subplot='polar'
        ))
        legend = False

    neuron_angles = np.arctan2(w[:,1], w[:,0]) * 180 / np.pi
    neuron_radii = rescale_magnitudes(w, r_min=min_radius, r_max=max_radius)

    # Determine deactivated neurons
    dot_products = np.dot(w, x.T)  # shape (m, n)
    deactivated_mask = np.all(dot_products <= 0, axis=1)  # shape (m,)

    # Plot deactivated neurons (gray)
    traces.append(go.Scatterpolar(
        r=neuron_radii[deactivated_mask],
        theta=neuron_angles[deactivated_mask],
        mode='markers',
        marker=dict(size=12, color='gray', opacity=0.5),
        name='Deactivated Neurons',
        showlegend=True,
    ))

    # Plot activated positive neurons (green)
    active_pos_mask = (~deactivated_mask) & np.isin(np.arange(len(w)), sPlus)
    traces.append(go.Scatterpolar(
        r=neuron_radii[active_pos_mask],
        theta=neuron_angles[active_pos_mask],
        mode='markers',
        marker=dict(size=12, color=color_neuron_pos, opacity=0.3),
        name='Positive Neurons',
        showlegend=True,
    ))

    # Plot activated negative neurons (red)
    active_neg_mask = (~deactivated_mask) & np.isin(np.arange(len(w)), sMinus)
    traces.append(go.Scatterpolar(
        r=neuron_radii[active_neg_mask],
        theta=neuron_angles[active_neg_mask],
        mode='markers',
        marker=dict(size=12, color=color_neuron_neg, opacity=0.3),
        name='Negative Neurons',
        showlegend=True,
    ))
    return traces

# --- Build frames ---
frames = []
for idx in frame_indices:
    a = current_a_values[idx]
    w = current_w_values[idx]
    y_pred = twoLayerReluNet(a, w, x_input)
    polar_traces = make_polar_traces(a, w)
    estimator_trace = go.Scatter(
        x=x1_range, y=y_pred, mode='lines', name='Estimator function', xaxis='x2', yaxis='y2',line=dict(color='blue')
    )
    dataset_trace = go.Scatter(
        x=x[:,0], y=y, mode='markers', name='Dataset', marker=dict(color='red', size=10, symbol='circle'), xaxis='x2', yaxis='y2'
    )
    frames.append(go.Frame(
        data=polar_traces + [estimator_trace, dataset_trace],
        name=str(idx)
    ))

# --- Initial data ---
a0 = current_a_values[frame_indices[0]]
w0 = current_w_values[frame_indices[0]]
y_pred0 = twoLayerReluNet(a0, w0, x_input)
initial_data = make_polar_traces(a0, w0) + [
    go.Scatter(
        x=x1_range, y=y_pred0, mode='lines', name='Estimator function', xaxis='x2', yaxis='y2', line=dict(color='blue')
    ),
    go.Scatter(
        x=x[:,0], y=y, mode='markers', name='Dataset', marker=dict(color='red', size=10, symbol='circle'), xaxis='x2', yaxis='y2'
    )
]

# --- Layout with subplots ---
layout = go.Layout(

    showlegend=True,
    polar=dict(
        domain=dict(x=[0, 0.48], y=[0, 1]),
        radialaxis=dict(
            range=[min_radius, max_radius],
            showticklabels=False,
            visible=True,
            gridcolor='gray',
            showgrid=True,
        ),
        angularaxis=dict(
            showticklabels=False,
            gridcolor='gray',
            showgrid=False
        )
    ),
    xaxis2=dict(
        domain=[0.52, 1],
        title=r'$x[1]$',
        range=[-2, 5]
    ),
    yaxis2=dict(
        title='Network output',
        range=[min(y)-1, max(y)+1]
    ),
    width=1650,
    height=800,
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]},
            {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }],
    sliders=[{
        "steps": [
            {"args": [[str(idx)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
             "label": "",
             "method": "animate"}
            for idx in frame_indices
        ],
        "transition": {"duration": 0},
        "x": 0.1,
        "len": 0.9,
        "y": -0.2,
        "currentvalue": {"visible": False},
        "tickwidth": 0,
        "tickcolor": "rgba(0,0,0,0)",
    }]
)

fig = go.Figure(data=initial_data, layout=layout, frames=frames)
fig.write_html("plotly_figures/merged_extremal_and_estimator_animation.html", include_plotlyjs="cdn", include_mathjax="cdn", auto_play=False)
from IPython.display import HTML
HTML(fig.to_html(include_plotlyjs="cdn", include_mathjax="cdn"))

# Zero Loss example

## Data and Training

In [ ]:
x = np.array([[1,1],[2,1],[4,1]])
y = np.array([1,4,2])

n = len(y)

m = 60
lambda_param = 1e-7
learning_rate = 1e-3
n_steps = 11000


train(n_steps, learning_rate, lambda_param,m)

In [ ]:
import plotly.graph_objects as go
import numpy as np

current_a_values = np.load('weights\\a_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))
current_w_values = np.load('weights\\w_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))

sPlus = np.load('weights\\sPlus_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))
sMinus = np.load('weights\\sMinus_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))

# --- Prepare data and frame indices ---
x1_range = np.linspace(-5, 5, 200)
x_input = np.stack([x1_range, np.ones_like(x1_range)], axis=1)
theta = np.linspace(0, 2 * np.pi, 400)
x_plot = np.cos(theta)
y_plot = np.sin(theta)
line_styles = ['dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
max_radius = 3
min_radius = -0.7

frame_indices = np.linspace(0, len(current_a_values)-1, 500, dtype=int)

def rescale_magnitudes(w, r_min=0.2, r_max=0.8):
    """
    Rescale the norms of w to be between r_min and r_max for polar plotting.
    """
    w_norms = np.linalg.norm(w, axis=1)
    
    return w_norms * 2.25

def make_polar_traces(a, w, pad_to=None):
    # Polar plot traces
    z = np.zeros(len(x_plot))
    for i in range(len(x_plot)):
        z[i] = funcG_non_zero(np.array([[x_plot[i], y_plot[i]]]), a, w)

    traces = []

    r_circle = [0.005] * 361
    theta_circle = np.linspace(0, 360, 361)
    traces.append(go.Scatterpolar(
        r=r_circle,
        theta=theta_circle,
        mode='lines',
        line=dict(color='black', width=3, dash='dot'),
        name='Radius=0',
        showlegend=True,
        opacity=1.0,
        subplot='polar'
    ))

    traces.append(go.Scatterpolar(
        r=z,
        theta=theta * 180 / np.pi,
        mode='lines',
        name='BF Function',
        line=dict(color=color_BF, width=2),
        subplot='polar'
    ))

    l = 1
    for idx, vec in enumerate(x):
        style = line_styles[idx % len(line_styles)]
        x_line_orth, y_line_orth = vec[0], vec[1]
        x1 = 1
        x2 = - x_line_orth / y_line_orth
        theta_line = np.degrees(np.arctan2(x2, x1))
        r_line = np.linspace(min_radius, max_radius, 100)
        theta_line_array = np.full_like(r_line, fill_value=theta_line)
        theta_opposite_array = np.full_like(r_line, fill_value=(theta_line + 180) % 360)
        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_line_array,
            mode='lines',
            line=dict(color=color_boundaries, width=0.8, dash='dash'),
            subplot='polar',
            showlegend=False
        ))
        l += 1
        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_opposite_array,
            mode='lines',
            line=dict(color=color_boundaries, width=0.8, dash='dash'),
            showlegend=False,
            subplot='polar',
        ))

    legend = True
    extremal_vectors = extremal_vectors_time(a, w)
    for vec in extremal_vectors:
        x1, x2 = vec[0], vec[1]
        theta_line = np.degrees(np.arctan2(x2, x1))
        r_line = np.linspace(min_radius, max_radius, 100)
        theta_line_array = np.full_like(r_line, fill_value=theta_line)
        traces.append(go.Scatterpolar(
            r=r_line,
            theta=theta_line_array,
            mode='lines',
            line=dict(color=color_extremal, width=2.5),
            name=f'Extremal vector',
            showlegend=legend,
            subplot='polar'
        ))
        legend = False

    neuron_angles = np.arctan2(w[:,1], w[:,0]) * 180 / np.pi
    neuron_radii = rescale_magnitudes(w, r_min=min_radius, r_max=max_radius)

    # Determine deactivated neurons
    dot_products = np.dot(w, x.T)  # shape (m, n)
    deactivated_mask = np.all(dot_products <= 0, axis=1)  # shape (m,)

    # Plot deactivated neurons (gray)
    traces.append(go.Scatterpolar(
        r=neuron_radii[deactivated_mask],
        theta=neuron_angles[deactivated_mask],
        mode='markers',
        marker=dict(size=12, color='gray', opacity=0.5),
        name='Deactivated Neurons',
        showlegend=True,
    ))

    # Plot activated positive neurons (green)
    active_pos_mask = (~deactivated_mask) & np.isin(np.arange(len(w)), sPlus)
    traces.append(go.Scatterpolar(
        r=neuron_radii[active_pos_mask],
        theta=neuron_angles[active_pos_mask],
        mode='markers',
        marker=dict(size=12, color=color_neuron_pos, opacity=0.3),
        name='Positive Neurons',
        showlegend=True,
    ))

    # Plot activated negative neurons (red)
    active_neg_mask = (~deactivated_mask) & np.isin(np.arange(len(w)), sMinus)
    traces.append(go.Scatterpolar(
        r=neuron_radii[active_neg_mask],
        theta=neuron_angles[active_neg_mask],
        mode='markers',
        marker=dict(size=12, color=color_neuron_neg, opacity=0.3),
        name='Negative Neurons',
        showlegend=True,
    ))

    # Padding to consistent trace count
    while len(traces) < pad_to:
        traces.append(go.Scatterpolar(r=[], theta=[], mode='lines', showlegend=False))
    return traces


max_trace_count = 80
# --- Build frames ---
frames = []
for idx in frame_indices:
    a = current_a_values[idx]
    w = current_w_values[idx]
    y_pred = twoLayerReluNet(a, w, x_input)
    polar_traces = make_polar_traces(a, w, pad_to=max_trace_count)
    estimator_trace = go.Scatter(
        x=x1_range, y=y_pred, mode='lines', name='Estimator function', xaxis='x2', yaxis='y2',line=dict(color='blue')
    )
    dataset_trace = go.Scatter(
        x=x[:,0], y=y, mode='markers', name='Dataset', marker=dict(color='red', size=10, symbol='circle'), xaxis='x2', yaxis='y2'
    )
    frames.append(go.Frame(
        data=polar_traces + [estimator_trace, dataset_trace],
        name=str(idx)
    ))

# --- Initial data ---
a0 = current_a_values[frame_indices[0]]
w0 = current_w_values[frame_indices[0]]
y_pred0 = twoLayerReluNet(a0, w0, x_input)

  # Set a maximum trace count for padding
# # Create frames
# frame_indices = np.linspace(0, len(current_a_values)-1, 500, dtype=int)  # 50 frames for speed
# frames = [
#     go.Frame(
#         data=make_frame(idx, pad_to=max_trace_count),
#         name=str(idx)
#     )
#     for idx in frame_indices
# ]

# Initial data
#initial_data = make_frame(frame_indices[0], pad_to=max_trace_count)

initial_data = make_polar_traces(a0, w0, pad_to=max_trace_count) + [
    go.Scatter(
        x=x1_range, y=y_pred0, mode='lines', name='Estimator function', xaxis='x2', yaxis='y2', line=dict(color='blue')
    ),
    go.Scatter(
        x=x[:,0], y=y, mode='markers', name='Dataset', marker=dict(color='red', size=10, symbol='circle'), xaxis='x2', yaxis='y2'
    )
]

# --- Layout with subplots ---
layout = go.Layout(

    showlegend=True,
    polar=dict(
        domain=dict(x=[0, 0.48], y=[0, 1]),
        radialaxis=dict(
            range=[min_radius, max_radius],
            showticklabels=False,
            visible=True,
            gridcolor='gray',
            showgrid=True,
        ),
        angularaxis=dict(
            showticklabels=False,
            gridcolor='gray',
            showgrid=False
        )
    ),
    xaxis2=dict(
        domain=[0.52, 1],
        title=r'$x[1]$',
        range=[-2, 5]
    ),
    yaxis2=dict(
        title='Network output',
        range=[min(y)-1.5, max(y)+1]
    ),
    width=1650,
    height=800,
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]},
            {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }],
    sliders=[{
        "steps": [
            {"args": [[str(idx)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
             "label": "",
             "method": "animate"}
            for idx in frame_indices
        ],
        "transition": {"duration": 0},
        "x": 0.1,
        "len": 0.9,
        "y": -0.2,
        "currentvalue": {"visible": False},
        "tickwidth": 0,
        "tickcolor": "rgba(0,0,0,0)",
    }]
)

fig = go.Figure(data=initial_data, layout=layout, frames=frames)
fig.write_html("plotly_figures/merged_extremal_and_estimator_animation_nonzero_loss.html", include_plotlyjs="cdn", include_mathjax="cdn", auto_play=False)
from IPython.display import HTML
HTML(fig.to_html(include_plotlyjs="cdn", include_mathjax="cdn"))

In [ ]:
import numpy as np
import itertools
import plotly.graph_objects as go
from IPython.display import HTML



current_a_values = np.load('weights\\a_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))
current_w_values = np.load('weights\\w_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))

sPlus = np.load('weights\\sPlus_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))
sMinus = np.load('weights\\sMinus_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))

# Generate permutations
permutations = [p for p in itertools.product([1, 0, -1], repeat=n) if sum(np.array(p) == 0) <= 1]

# Shading function
def compute_shaded_area(permutation):
    u = np.array(permutation)
    x_vals = np.linspace(-1, 1, 400)
    y_vals = np.linspace(-1, 1, 400)
    X, Y = np.meshgrid(x_vals, y_vals)
    feasible = np.ones((400, 400), dtype=bool)

    for i in range(len(x)):
        w1, w2 = x[i]
        if u[i] == 1:
            feasible &= (w1 * X + w2 * Y >= 0)
        elif u[i] == -1:
            feasible &= (w1 * X + w2 * Y <= 0)

    rotation = np.rot90(feasible, 2)
    shaded_area = np.maximum(feasible, rotation)
    return x_vals, y_vals, shaded_area.astype(int)

# Trace builder
def compute_vector_trace(permutation, pad_to=None, index=3446):
    
    y_x = (y - twoLayerReluNet(current_a_values[index], current_w_values[index], x))[:, np.newaxis] * x    

    traces = []

    # Decision boundaries
    for vec in x:
        w1, w2 = vec
        x_vals = np.linspace(-1, 1, 100)
        y_vals = - (w1 * x_vals) / w2
        traces.append(go.Scatter(
            x=x_vals,
            y=y_vals,
            mode="lines",
            line=dict(color="gray", width=1),
            showlegend=False,
            name="Activation boundary"
        ))

    u = np.array(permutation)
    selected_elements = y_x[u == 1]
    sum_vector = np.sum(selected_elements, axis=0)
    norm_vec = sum_vector / (1 + np.linalg.norm(sum_vector))

    if not np.allclose(norm_vec, 0):
        traces.append(go.Scatter(
            x=[0, norm_vec[0]],
            y=[0, norm_vec[1]],
            mode="lines+markers",
            line=dict(color=color_extremal, width=2),
            marker=dict(size=10, symbol="arrow-bar-up", angleref="previous"),
            name="Gradient vector",
            showlegend=False
        ))
    else:
        traces.append(go.Scatter(x=[], y=[], mode="lines"))  # placeholder

    # Extra vector if there's a zero
    if np.any(u == 0):
        index = np.where(u == 0)[0][0]
        add_vector = sum_vector + y_x[index]
        norm_add = add_vector / (1 + np.linalg.norm(add_vector))

        traces.append(go.Scatter(
            x=[norm_vec[0], norm_add[0]],
            y=[norm_vec[1], norm_add[1]],
            mode="lines",
            line=dict(color=color_extremal, width=1.5, dash="dash"),
            name="Range of gradients",
            showlegend=False
        ))

        traces.append(go.Scatter(
            x=[0, norm_add[0]],
            y=[0, norm_add[1]],
            mode="lines+markers",
            line=dict(color=color_extremal, width=2),
            marker=dict(size=10, symbol="arrow-bar-up", angleref="previous"),
            name="Gradient vector 2",
            showlegend=False
        ))
    else:
        traces += [go.Scatter(x=[], y=[], mode="lines")] * 2  # placeholders

    # Shaded area
    x_vals, y_vals, shaded_area = compute_shaded_area(permutation)

    if np.any(u == 0):
        shaded_area = np.zeros((400, 400), dtype=int)
        index = np.where(u == 0)[0][0]
        w1, w2 = x[index]
        x_vals = np.linspace(-1, 1, 100)
        y_vals = - (w1 * x_vals) / w2
        traces.append(go.Scatter(
            x=x_vals,
            y=y_vals,
            mode="lines",
            line=dict(color="green", width=3),
            showlegend=False,
            name="Feasible boundary"
        ))
    else:
        traces.append(go.Scatter(x=[], y=[], mode="lines"))  # placeholder

    traces.append(go.Heatmap(
        z=shaded_area,
        x=np.linspace(-1, 1, 400),
        y=np.linspace(-1, 1, 400),
        colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,128,0,0.2)"]],
        showscale=False,
        hoverinfo="none",
        zmin=0,
        zmax=1
    ))

    # Padding to consistent trace count
    if pad_to:
        while len(traces) < pad_to:
            traces.append(go.Scatter(x=[], y=[], mode="lines"))

    return traces

# Set high max trace count to plot the whole graph
max_trace_count = 50

frames = [
    go.Frame(
        data=compute_vector_trace(p, pad_to=max_trace_count),
        name=str(p),
        layout=go.Layout(title=dict(
                text=rf"$\text{{Determining if an extremal vector exists for activation }} u = {tuple(p)}$",
                x=0.5,           # Center the title
                xanchor='center' # Anchor at the center
            ),
        )
    ) for p in permutations
]


# Initial frame
initial_data = compute_vector_trace(permutations[0], pad_to=max_trace_count)

# Dropdown
dropdown_buttons = [{
    "label": str(p),
    "method": "animate",
    "args": [[str(p)], {"frame": {"duration": 0}, "mode": "immediate"}]
} for p in permutations]

# Final figure
fig = go.Figure(
    data=initial_data,
    layout=go.Layout(
        title=dict(
            text=rf"Determining extremal vectors",
            x=0.5,           # Center the title
            xanchor='center' # Anchor at the center
        ),
        xaxis=dict(range=[-1, 1], showticklabels=False, title=dict(text=r"$w[1]$")),
        yaxis=dict(range=[-1, 1], showticklabels=False, title=dict(text=r"$w[2]$")),
        width=700,
        height=700,
        updatemenus=[{
            "buttons": dropdown_buttons,
            "direction": "down",
            "showactive": True,
            "x": 1.1,
            "xanchor": "left",
            "y": 1.0,
            "yanchor": "top"
        }]
    ),
    frames=frames
)

# Export & show
fig.write_html("plotly_figures/activation_vectors_dropdown.html", include_plotlyjs="cdn",include_mathjax="cdn", auto_play=False)
HTML(fig.to_html(include_plotlyjs="cdn",include_mathjax="cdn"))


In [ ]:
x = np.array([[-0.8,1],[-0.1,1],[0.9,1]])
y = np.array([1.1,0.1,1])

n = len(y)

m = 60
lambda_param = 1e-6
learning_rate = 1e-3
n_steps = 25000


train(n_steps, learning_rate, lambda_param,m)

import plotly.graph_objects as go
import numpy as np

current_a_values = np.load('weights\\a_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))
current_w_values = np.load('weights\\w_values_m%d_Learning%d_Steps%d.npy' % (m, learning_rate, n_steps))

# Define the input range for the first coordinate
x1_range = np.linspace(-5, 5, 200)
x_input = np.stack([x1_range, np.ones_like(x1_range)], axis=1)

# Prepare frames for animation
frame_indices = np.linspace(0, len(current_a_values)-1, 500, dtype=int)  # 100 frames for speed
frames = []
for idx in frame_indices:
    a = current_a_values[idx]
    w = current_w_values[idx]
    y_pred = twoLayerReluNet(a, w, x_input)
    frames.append(go.Frame(
        data=[
            go.Scatter(x=x1_range, y=y_pred, mode='lines', name='Estimator function', line=dict(color='blue')),
            go.Scatter(x=x[:,0], y=y, mode='markers', name='Dataset', marker=dict(color='red', size=10, symbol='circle'))
        ],
        name=str(idx)
    ))

# Initial frame
a0 = current_a_values[frame_indices[0]]
w0 = current_w_values[frame_indices[0]]
y_pred0 = twoLayerReluNet(a0, w0, x_input)

fig = go.Figure(
    data=[
        go.Scatter(x=x1_range, y=y_pred0, mode='lines', name='Estimator function', line=dict(color='blue')),
        go.Scatter(x=x[:,0], y=y, mode='markers', name='Dataset', marker=dict(color='red', size=10, symbol='circle'))
    ],
    layout=go.Layout(
        # title=dict(
        #     text='Network output over time',
        #     x=0.5,
        #     xanchor='center'
        # ),
        xaxis=dict(
            title=r'$x[1]$',
            range=[-2, 2]
        ),
        yaxis=dict(
            title='Network output',
            range=[-0.5, 1.5]  # Adjust as needed for your data
        ),
        width=1650,
        height=800,
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]},
                {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }],
        sliders=[{
            "steps": [
                {"args": [[str(idx)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                 "label": "",
                 "method": "animate"}
                for idx in frame_indices
            ],
            "transition": {"duration": 0},
            "x": 0.1,
            "len": 0.9,
            "y": -0.2,  # Lower the slider (try values like -0.1, -0.2, etc.)
            "currentvalue": {"visible": False},
            "tickwidth": 0,
            "tickcolor": "rgba(0,0,0,0)",
            
        }]
    ),
    frames=frames
)

fig.write_html("plotly_figures/BF_least_square_estimator_example.html", include_plotlyjs="cdn",include_mathjax="cdn", auto_play=False)
HTML(fig.to_html(include_plotlyjs="cdn",include_mathjax="cdn"))